In [53]:
import time
import requests
import pandas as pd

from datetime import datetime
from tqdm.notebook import tqdm
from user_agent import generate_user_agent


#### Collect Data

In [33]:
user_agents = generate_user_agent()
api_key = "RGAPI-05a04f36-b0e0-47f1-b9c6-22dfe97bee09"
header = {
          "User-Agent" : user_agents,
          "Accept-Language": "ko-KR,ko;q=0.9,en;q=0.8",
          "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
          "Origin": "https://developer.riotgames.com",
          "X-Riot-Token": api_key
}

In [34]:
def league_v4(tier):
    queue = "RANKED_SOLO_5x5"

    if tier == "challenger":
        url = f"https://kr.api.riotgames.com/lol/league/v4/{tier}leagues/by-queue/{queue}"
    elif tier == "grandmaster":
        url = f"https://kr.api.riotgames.com/lol/league/v4/{tier}leagues/by-queue/{queue}"
    else:
        url = f"https://kr.api.riotgames.com/lol/league/v4/{tier}leagues/by-queue/{queue}"

    return requests.get(url, headers=header)

In [35]:
req = league_v4("master")

if req.status_code == 200:
    data = req.json()["entries"]

    summonerId = [item["summonerId"] for item in data]
else:
    print(req.status_code)

len(summonerId)

15822

#### 소환사 범위 정하기

In [36]:
resize_summonerId = summonerId[-200:-100]
len(resize_summonerId)

100

In [37]:
def summoner_v4(encryptedSummonerId):
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{encryptedSummonerId}"

    return requests.get(url, headers=header)

In [39]:
puuId = []
for _summonerId in tqdm(resize_summonerId):
    req = summoner_v4(_summonerId)

    if req.status_code == 200:
        data = req.json()

        puuId.append(data["puuid"])
    else:
        print(req.status_code)
    time.sleep(1.3)
len(puuId)

  0%|          | 0/100 [00:00<?, ?it/s]

100

In [49]:
def match_v5(puuid, startTime, endTime, type, start, count):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={startTime}&endTime={endTime}&type={type}&start={start}&count={count}"
    
    return requests.get(url, headers=header)

In [50]:
startTime = int(datetime(2023, 2, 1, 0, 0, 0).timestamp())
endTime = int(datetime(2023, 12, 31, 23, 59, 59).timestamp())
type = 'ranked'
startTime, endTime

(1675177200, 1704034799)

In [45]:
matchId = []

for _puuId in tqdm(puuId):
    req = match_v5(_puuId, startTime, endTime, type, 0, 100)

    if req.status_code == 200:
        data = req.json()
        matchId.append(data)
    else:
        print(req.status_code)
    time.sleep(1.3)
len(matchId), print(matchId)

  0%|          | 0/100 [00:00<?, ?it/s]

[['KR_6868260905', 'KR_6868227887', 'KR_6868221050', 'KR_6868176888', 'KR_6860870317', 'KR_6860858057', 'KR_6860831909', 'KR_6860828904', 'KR_6860832065', 'KR_6854264498', 'KR_6854202525', 'KR_6854185512', 'KR_6854157738', 'KR_6854142146', 'KR_6854121829', 'KR_6853732962', 'KR_6853678717', 'KR_6853595936', 'KR_6853512658', 'KR_6853380314', 'KR_6853282512', 'KR_6853078076', 'KR_6853026600', 'KR_6852973912', 'KR_6852736668', 'KR_6852649089', 'KR_6852581364', 'KR_6852137365', 'KR_6852070144', 'KR_6851963122', 'KR_6851872049', 'KR_6851854168', 'KR_6850712150', 'KR_6850600387', 'KR_6850420654', 'KR_6850218780', 'KR_6850144444', 'KR_6849952382', 'KR_6849877080', 'KR_6849826822', 'KR_6849797858', 'KR_6849777831', 'KR_6849774070', 'KR_6829476505', 'KR_6829405164', 'KR_6829294806', 'KR_6829231401', 'KR_6824054456', 'KR_6823974817', 'KR_6823968276', 'KR_6819178484', 'KR_6819151014', 'KR_6819120148', 'KR_6818812924', 'KR_6818804390', 'KR_6818796424', 'KR_6808239037', 'KR_6808182361', 'KR_68081502

(100, None)

In [51]:
def matchData(matchId):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}"

    return requests.get(url, headers=header)

In [47]:
match_data = []
for matchIds in tqdm(matchId):
    for _matchId in matchIds:
        req = matchData(_matchId)
        if req.status_code == 200:
            data = req.json()
            match_data.append(data)
        else:
            print(req.status_code)
            break
        time.sleep(1.3)
len(match_data)

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [24]:
len(match_data)

7226

#### DataFrame (수정예정)

In [25]:
Keywords = [
    'puuid',
    'summonerName',
    'individualPosition',
    'championId',
    'championName',

    ###### KDA
    'kills',
    'deaths',
    'assists',

    ###### Pings
    'commandPings',
    'assistMePings',
    'enemyMissingPings',
    'getBackPings',
    'dangerPings',
    'onMyWayPings',

    ####### vision
    'visionScore',

    'win'
]

challenge_Keywords = [
    'stealthWardsPlaced',
    'controlWardsPlaced',
]

team_keywords = [
    'baron',
    'dragon',
    'horde',
    'riftHerald',
    'tower'
]

In [40]:
all_match_result = []
df_all_match_result = pd.DataFrame(all_match_result)

for n in tqdm(range(len(match_data))) :

    match_result = []
    match_list = match_data[n]
    match_info = match_list['info']


    for each in Keywords :

        match_user_data = [participant[each] for participant in match_info['participants']]
        match_result.append(match_user_data)

    for each in challenge_Keywords :
        match_ward_data = []
        for i in range(0,10) :
            data = match_info['participants'][i]['challenges'][each]
            match_ward_data.append(data)
        match_result.append(match_ward_data)

    for each in team_keywords :
        if each in match_info['teams'][0]['objectives'] :
            match_team_data = [match_info['teams'][0]['objectives'][each]['first']]*5 + [match_info['teams'][1]['objectives'][each]['first']]*5
        else :
            match_team_data = [None]*10
        match_result.append(match_team_data)

    df_match_result = pd.DataFrame(match_result)
    df_all_match_result = pd.concat([df_all_match_result,df_match_result], axis=1)

0it [00:00, ?it/s]

In [41]:
DF = df_all_match_result.transpose()
DF.reset_index(drop=True, inplace=True)
DF.columns = Keywords + challenge_Keywords + team_keywords
DF.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 23 elements